# 1. Import các thư viện cần thiết

In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

import os
from playsound import playsound
import speech_recognition as sr
import time
import sys
import ctypes
import wikipedia
import datetime
import json
import re
import webbrowser
import smtplib
import requests
import urllib
import urllib.request as urllib2
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from time import strftime
from gtts import gTTS
from youtube_search import YoutubeSearch

import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

nltk.download('punkt')
# select "model" - "punkt" - "download"

import numpy as np
import random

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\phamd\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# 2. Khai báo biến mặc định

In [2]:
wikipedia.set_lang('vi')
language = 'vi'
path = ChromeDriverManager().install()



====== WebDriver manager ======
Current google-chrome version is 94.0.4606
Get LATEST driver version for 94.0.4606
There is no [win32] chromedriver for browser 94.0.4606 in cache
Get LATEST driver version for 94.0.4606
Trying to download new driver from https://chromedriver.storage.googleapis.com/94.0.4606.61/chromedriver_win32.zip
Driver has been saved in cache [C:\Users\phamd\.wdm\drivers\chromedriver\win32\94.0.4606.61]


# 3. Chức năng chuyển văn bản thành âm thanh

  -  Dùng `gTSS (google Text To Speech)` chuyển văn bản thành âm thanh theo tiếng Việt và lưu lại dưới dạng file **sound.mp3**
  -  Sau đó dùng hàm `playsound.playsound()` để đọc file **sound.mp3** trên máy tính
  -  Xóa file **sound.mp3** sau khi đọc xong để tránh lỗi trùng lắp file khi đọc nhiều văn bản

In [3]:
def speak(text):
    print("Vias: {}".format(text))
    tts = gTTS(text=text, lang='vi', slow=False)
    date_string = datetime.datetime.now().strftime("%d%m%Y%H%M%S")
    filename = "voice"+date_string+".mp3"
    tts.save(filename)
    playsound(filename) 
    os.remove(filename)

    

def speak_other_language(text, otherLanguage):
    print("Vietvias: {}".format(text))
    tts = gTTS(text=text, lang = otherLanguage, slow=False)
    date_string = datetime.datetime.now().strftime("%d%m%Y%H%M%S")
    filename = "voice"+date_string+".mp3"
    tts.save(filename)
    playsound(filename) 
    os.remove(filename)

# 4. Chức năng chuyển âm thanh thành văn bản
   Trong chức năng này, dùng 2 hàm là `get_audio()`, `stop()` và `get_text()`.
   
   
### Trong hàm `get_audio()`
 -  Dùng thư viện speech_recognition có chức năng nhân diện giọng nói để chuyển âm thanh thành văn bản.
 -  Âm thanh được đọc vào sau đó được xử lý qua hàm `listen()` và lưu vào biến **audio**.
 -  Âm thanh sẽ được nhận dạng ở tiếng Việt bằng hàm `recognize_google()` và lưu kết quả vào biến **text**
 - Nếu **audio** không lỗi tức `recognize_google()` có thể nhận dạng được thì hàm sẽ trả về **text**. Ngược lại, nếu lỗi thì sẽ trả về **0**.
 
  ### Trong hàm `stop()`
   Đọc đoạn text "Tạm biệt" bằng cách dùng hàm `speak()` ở trên.
   
  ### Trong hàm `get_text()`
 - Máy tính sẽ nhận dạng âm thanh của người đọc tối đa 3 lần (bằng cách dùng for 3 lần), nếu text khác **0** thì `get_text()` trả vè `text.lower()`. 
 - Ngược lại text = 0 thì trả về hàm `stop()` và `get_text()` trả về 0
 - `time.sleep(3)`: tạm dừng 3s tránh máy đọc các đoạn văn bản bị khớp nhau. 


In [4]:
def get_audio():
    r = sr.Recognizer()
    with sr.Microphone() as source:

        print("Tôi: ", end='')
        audio = r.record(source, duration=5)
        try:
            text = r.recognize_google(audio, language="vi-VN")
            print(text)
            return text
        except:
            print("...")
            return 0
        
def stop():
    speak("Tạm biệt")
    
def get_text():
    for i in range(3):
        text = get_audio()
        if text:
            return text.lower()
        elif i < 2:
            speak("Vias không nghe rõ. Bạn nói lại được không!")
    time.sleep(3)
    stop()
    return 0

# 5. Chức năng hiển thị thời gian
- Dùng thư viện **`datetime`**, lưu vào biến **now**
- Nếu trong biến **text** đầu vào có từ `"giờ"` thì sẽ đọc thời gian hiện tại trong ngày
- Nếu trong biến **text** đầu vào có từ `"ngày"` thì sẽ đọc thời điểm hiện tại trong năm

In [5]:
def get_time(text):
    now = datetime.datetime.now()
    if "giờ" in text:
        speak('Bây giờ là %d giờ %d phút' % (now.hour, now.minute))
    elif "ngày" in text:
        speak("Hôm nay là ngày %d tháng %d năm %d" %
              (now.day, now.month, now.year))
    else:
        speak("Vias chưa hiểu ý của bạn. Bạn nói lại được không?")

# 6. Chức năng mở ứng dụng hệ thống, website và chức năng tìm kiếm từ khóa trên Google
### Hàm `open_application()`
- Dùng hàm `os.startfile()` để mở các file ứng dụng từ hệ thống

### Hàm `open_website()`
- Dùng hàm `re.search()` để tách "domain" sau từ "mở" trong biến **text** và ghép nó với phần tiền tố "https://www." để tạo thành **url** của web.
- Tiếp theo, dùng `webbroser.open(url)` để mở trang web theo url
- Nếu domain được tìm thấy thì thực hiện mở website bằng `open_website()` và trả về **True**, ngược lại thì không thực hiện gì cả và trả về **False**.

### Hàm `open_google_and_search()`
- Tách từ khóa sau từ "kiếm" trong **text** bằng `split()` rồi lưu vào **search_for**
- Gọi hàm `webdriver.Chrome(path)` để mở Google Chrome 
- Dùng hàm `driver.find_element_by_path()` để lấy thẻ **query** (viết tắt là q) rồi lưu vào biến que
- Biến **que** thực hiện tiềm kiếm từ khóa **search_for** và trả trên Google Search. 

In [6]:
def open_application(text):
    if "google" in text:
        speak("Mở Google Chrome")
        os.startfile('C:\Program Files (x86)\Google\Chrome\Application\chrome.exe')
    elif "word" in text:
        speak("Mở Microsoft Word")
        os.startfile('C:\Program Files\Microsoft Office\\root\Office16\\WINWORD.EXE')
    elif "excel" in text:
        speak("Mở Microsoft Excel")
        os.startfile('C:\Program Files\Microsoft Office\\root\Office16\EXCEL.EXE')
    else:
        speak("Ứng dụng chưa được cài đặt. Bạn hãy thử lại!")
        
def open_website(text):
    reg_ex = re.search('mở (.+)', text)
    if reg_ex:
        domain = reg_ex.group(1)
        url = 'https://www.' + domain
        webbrowser.open(url)
        speak("Trang web bạn yêu cầu đã được mở.")
        return True
    else:
        return False
    
def open_google_and_search(text):
    search_for = text.split("kiếm", 1)[1]
    speak('Okay!')
    driver = webdriver.Chrome(path)
    driver.get("http://www.google.com")
    que = driver.find_element_by_xpath("//input[@name='q']")
    que.send_keys(str(search_for))
    que.send_keys(Keys.RETURN)

# 7. Chức năng gửi Email (chưa hoàn thiện)
- Sử dụng thư viện **`smtplib`** để thực hiện gửi mail bằng stmp.
- SMTP - Simple Mail Transfer Protocol: Giao thức truyền tải thư tín đơn giản
- Lấy tên người nhận bằng hàm `get_text()`, lưu vào biến **recipient**
    vd: tìm thấy tên "thục" trong tên người cần gửi thì thực hiện gửi Email.
- Gọi `get_text()` để lấy nội dung cần gửi rồi lưu vào biến **content**
- Tiếp tục mở đường truyền gửi Email băng `stmp` rồi đăng nhập tài khoản Gmail của mình
- Kết nối thành công, sẽ gửi email từ địa chỉ đã đăng nhập đến địa chỉ người nhận (dinhthucf12@gmail.com) với nội dung **content** rồi đóng đường truyền

In [7]:
def send_email(text):
    speak('Bạn gửi email cho ai?')
    recipient = get_text()
    speak("Xác nhận người nhận: " + recipient)
    speak("Yes or No ?")
    checkUser = get_text()
    if 'yes' in checkUser:
        speak('Nội dung bạn muốn gửi là gì')
        content = get_text()
        mail = smtplib.SMTP('smtp.gmail.com', 587)
        mail.ehlo()
        mail.starttls()
        mail.login('phamdinhthuc020100@gmail.com', 'dinhthuc123')
        mail.sendmail('phamdinhthuc020100@gmail.com',
                      'dinhthucf12@gmail.com', content.encode('utf-8'))
        mail.close()
        speak('Email của bạn vùa được gửi. Bạn check lại email nhé.')
    else:
        speak('Không xác nhận người nhận mail')

# 8. Chức năng xem dự báo thời tiết
- Dùng **Open WeatherMap** tại trang web `openweathermap.org`
- Dùng biến ow_url để lưu đường dẫn đến api của trang web
- Gọi get_text() để lấy thông tin thành phố cần xem thông tin thời tiết lưu vào biến **city**. Nếu không nghe được tên thì sẽ bỏ qua
- Kết nối với api của trang web (cần đăng ký tài khoản để lấy api_key)
- Biến **call_url** sẽ lưu đường dẫn đầy đủ để truy vấn bao gồm thông tin tên thành phố **city**
- Gọi `requests.get(call_url)` lấy thông tin truy vấn, lưu vào **response**
- `response.json()` sẽ chuyển dữ liệu về json rồi lưu vào **data**
- Nếu data["cod"] không trả về 404 tức requests không lỗi, thực hiện chức năng
- Từ **data** lấy các thông tin: **temperature**, **pressure**, **humidity**
- Lưu các kết quả vào biến **content** và đọc kết quả
- time.sleep(20): tạm dừng 20s để đọc **content**


In [8]:
def current_weather():
    speak("Bạn muốn xem thời tiết ở đâu?")
    ow_url = "http://api.openweathermap.org/data/2.5/weather?"
    city = get_text()
    if not city:
        pass
    api_key = "fe8d8c65cf345889139d8e545f57819a"
    call_url = ow_url + "appid=" + api_key + "&q=" + city + "&units=metric"
    response = requests.get(call_url)
    data = response.json()
    if data["cod"] != "404":
        city_res = data["main"]
        current_temperature = city_res["temp"]
        current_pressure = city_res["pressure"]
        current_humidity = city_res["humidity"]
        suntime = data["sys"]
        sunrise = datetime.datetime.fromtimestamp(suntime["sunrise"])
        sunset = datetime.datetime.fromtimestamp(suntime["sunset"])
        wthr = data["weather"]
        weather_description = wthr[0]["description"]
        now = datetime.datetime.now()
        content = """
        Hôm nay là ngày {day} tháng {month} năm {year}
        Mặt trời mọc vào {hourrise} giờ {minrise} phút
        Mặt trời lặn vào {hourset} giờ {minset} phút
        Nhiệt độ trung bình là {temp} độ C
        Áp suất không khí là {pressure} héc tơ Pascal
        Độ ẩm là {humidity}%.""".format(day = now.day,month = now.month, year= now.year, hourrise = sunrise.hour, minrise = sunrise.minute,
                                                                           hourset = sunset.hour, minset = sunset.minute, 
                                                                           temp = current_temperature, pressure = current_pressure, humidity = current_humidity)
        speak(content)
        time.sleep(20)
    else:
        speak("Không tìm thấy địa chỉ của bạn")

# 9. Chức năng phát video trên Youtube
- Gọi `get_text()` lấy thông tin tên bài hát, lưu vào **myTitlle**
- Gọi While để tìm kết quả (bỏ qua kết nối yếu, ... )
- Biến **url** lưu đường dẫn kế quả đầu tiên tìm thấy trên Youtube
- Dùng hàm `webbrowser.open(url)` để mở đường dẫn đến video tìm được trên Google Chrome để phát nhạc.

In [9]:
def youtube():
    speak('Bạn muốn xem gì?')
    myTitlle = get_text()
    while True:
        result = YoutubeSearch(myTitlle, max_results=10).to_dict()
        if result:
            break
    url = 'https://www.youtube.com/watch?v=' + result[0]['id']
    webbrowser.open(url)
    speak("Video đang được mở trên Youtube.")

# 10. Chức năng thay đổi hình nền máy tính
- Dùng **Unsplash** tại trang web `unsplash.com`
- Đăng ký tài khoản để lấy api_key
- Biến **url** lưu đường dẫn đến api của trang web
- Mở url và lấy requests lưu vào biến **json_string**, ảnh sẽ được load, đọc và lưu vào biến **photo**
- **photo** sẽ được lưu lại thành file ảnh trong máy
- Dùng lệnh thay đổi hình nền của máy tính thông qua hàm `ctypes.windll.user32.SystemParametersInfoW()`

In [10]:
def change_wallpaper():
    api_key = 'RF3LyUUIyogjCpQwlf-zjzCf1JdvRwb--SLV6iCzOxw'
    url = 'https://api.unsplash.com/photos/random?client_id=' + \
        api_key  # pic from unspalsh.com
    f = urllib2.urlopen(url)
    json_string = f.read()
    f.close()
    parsed_json = json.loads(json_string)
    photo = parsed_json['urls']['full']
    # Location where we download the image to.
    urllib2.urlretrieve(photo, "C:/Users/Night Fury/Downloads/a.png")
    image=os.path.join("C:/Users/Night Fury/Downloads/a.png")
    ctypes.windll.user32.SystemParametersInfoW(20,0,image,3)
    speak('Hình nền máy tính vừa được thay đổi ngẫu nhiên')

# 11. Chức năng đọc báo ngày hôm nay
- Chọn **Newsapi** tại `newsapi.org`, đăng ký tài khoản lấy apiKey
- Lấy thông tin về chủ đề muốn đọc, dùng requests
- Kết quả thu được kiểu json lưu vào biến **api_response**
- Hiển thị 20 tin tức thu thập và mở đường dẫn đến 3 bài báo đầu tiên.

In [11]:
def read_news():
    speak("Bạn muốn đọc báo về gì?")
    
    queue = get_text()
    params = {
        'apiKey': '30d02d187f7140faacf9ccd27a1441ad',
        "q": queue,
    }
    api_result = requests.get('http://newsapi.org/v2/top-headlines?', params)
    api_response = api_result.json()
    print("Tin tức")

    for number, result in enumerate(api_response['articles'], start=1):
        print(f"""Tin {number}:\nTiêu đề: {result['title']}\nTrích dẫn: {result['description']}\nLink: {result['url']}
    """)
        if number <= 3:
            webbrowser.open(result['url'])

# 12. Chức năng tìm định nghĩa trên từ điển Wikipedia
- Dùng `get_text()` lấy thông tin về thứ muốn định nghĩa, lưu vào **text**
- Gọi hàm `wikipedia.summary(text).split('\n')` để lưu lại thành 1 list các đoạn nội dung mà wikipedia tìm được
- Đọc đoạn định nghĩa đầu tiên
- Máy sẽ hỏi bạn muốn nghe thêm hay không, nếu có máy sẽ đọc tiếp phần tiếp theo, nếu không thì sẽ dừng đọc

In [12]:
def tell_me_about():
    try:
        speak("Bạn muốn định nghĩa về gì?")
        text = get_text()
        contents = wikipedia.summary(text).split('\n')
        speak("Đang tìm kiếm định nghĩa.Bạn vui lòng chờ một lát")
        speak(contents[0])
        time.sleep(10)
        for content in contents[1:]:
            speak("Bạn muốn nghe thêm không")
            ans = get_text()
            if "có" not in ans:
                break    
            speak(content)
            time.sleep(10)

        speak('Cảm ơn bạn đã lắng nghe!!!')
    except:
        speak("Vias không định nghĩa được thuật ngữ của bạn. Xin mời bạn nói lại")

# 13. Chức năng giao tiếp, chào hỏi
- Chức năng giao tiếp thông thường giữa người và máy.

In [13]:
def tokenize(sentence):
        return nltk.word_tokenize(sentence)
def stem(word):
        return stemmer.stem(word.lower())

def bag_of_words(s, words):
    # stem each word
    sentence_words = [stem(word) for word in s]
    # initialize bag with 0 for each word
    bag = np.zeros(len(words), dtype=np.float32)
    for idx, w in enumerate(words):
        if w in sentence_words: 
            bag[idx] = 1

    return bag

In [14]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.l2 = nn.Linear(hidden_size, hidden_size) 
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax at the end
        return out

In [15]:
# Version 2.0 - AI ChatBot
with open('intents.json', 'r') as f:
    intents = json.load(f)

all_words = []
tags = []
xy = []
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    tag = intent['tag']
    # add to tag list
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag))

# stem and lower each word
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
# remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

# print(len(xy), "patterns")
# print(len(tags), "tags:", tags)
# print(len(all_words), "unique stemmed words:", all_words)

# create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)

X_train = np.array(X_train)
y_train = np.array(y_train)

#hyperparameters
batch_size = 8
hidden_size = 8
output_size = len(tags)
input_size = len(X_train[0])
learning_rate = 0.001
num_epochs = 1000

class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples
        
dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') # AMD not NVIDIA
model = NeuralNet(input_size, hidden_size, output_size).to(device)

# loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

for epoch in range(num_epochs):
    for(words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)

        # forward
        outputs = model(words)
        loss = criterion(outputs, labels)

        # backward and optimizer step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch + 1) % 100 == 0:
            print(f'epoch {epoch+1}/{num_epochs}, loss={loss.item():.4f}')
print(f'final loss, loss={loss.item():.4f}')

epoch 100/1000, loss=0.7673
epoch 200/1000, loss=0.2067
epoch 300/1000, loss=0.0046
epoch 400/1000, loss=0.0048
epoch 500/1000, loss=0.0030
epoch 600/1000, loss=0.0028
epoch 700/1000, loss=0.0010
epoch 800/1000, loss=0.0017
epoch 900/1000, loss=0.0002
epoch 1000/1000, loss=0.0005
final loss, loss=0.0005


In [16]:
data = {
    "model_state": model.state_dict(),
    "input_size": input_size,
    "output_size": output_size,
    "hidden_size": hidden_size,
    "all_words": all_words,
    "tags": tags
}

FILE = "data.pth"
torch.save(data, FILE)

print(f'training complete. Save to {FILE}')

training complete. Save to data.pth


In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open('intents.json', 'r') as json_data:
    intents = json.load(json_data)

FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

def chat():
  speak("Bắt đầu trò chuyện")
  print("Let's talk! (say 'Kết thúc' or 'stop' to exit)")
  while True:
      # sentence = "do you use credit cards?"
      sentence = get_text()
      if "kết thúc" in sentence or 'stop' in sentence:
          break

      sentence = tokenize(sentence)
      X = bag_of_words(sentence, all_words)
      X = X.reshape(1, X.shape[0])
      X = torch.from_numpy(X).to(device)

      output = model(X)
      _, predicted = torch.max(output, dim=1)

      tag = tags[predicted.item()]

      probs = torch.softmax(output, dim=1)
      prob = probs[0][predicted.item()]
      if prob.item() > 0.75:
          for intent in intents['intents']:
              if tag == intent["tag"]:
                  speak_other_language(random.choice(intent['responses']), "en")
      else:
          speak_other_language("I do not understand...", "en")



# 14. Chức năng hiển thị các khả năng của trợ lý ảo


In [18]:
def help_me():
    speak("""Vias có thể giúp bạn thực hiện các câu lệnh sau đây:
    1. Chào hỏi tiếng Anh
    2. Hiển thị giờ
    3. Mở website, application
    4. Tìm kiếm trên Google
    5. Gửi email
    6. Dự báo thời tiết
    7. Nghe nhạc
    8. Thay đổi hình nền máy tính
    9. Đọc báo hôm nay
    10. Kể bạn biết về thế giới """)

# 15. Kết hợp tất cả chức năng Trợ lý ảo Tiếng Việt 
- Dùng các cặp lệnh `if...elif...else` và `while True` để lặp vô hạn cho đến khi bạn muốn dừng
- `get_text()` để lấy thông tin yêu cầu người dùng, lưu vào **text**, nếu **text** nhận 0 tức máy không nghe được, máy sẽ break
- Nếu **text** nhận được giá trị, sẽ thực hiện các câu lệnh, khi nghe thấy "dừng lại" hoặc "tạm biệt" thì cũng sẽ dừng.


In [19]:
def assistant():
    speak("""Xin chào, Tôi là Vias
            Bạn tên là gì?""")
    name = get_text()
    if name:
        speak("Chào {}".format(name))
        speak("Tôi có thể giúp gì?")
        while True:
            text = get_text()
            if not text:
                break
            elif "goodbye" in text or "tạm biệt" in text or "kết thúc" in text or "stop" in text:
                stop()
                break
            elif "chức năng" in text:
                help_me()
            elif "chat" in text:
                chat()
            elif "giờ" in text or "ngày" in text:
                get_time(text)
            elif "tìm kiếm" in text:
                open_google_and_search(text)
            elif "mở" in text:
                if "." in text:
                    open_website(text)
                elif "app" in text:
                    open_application(text)
            elif "email" in text or "mail" in text or "gmail" in text:
                send_email(text)
            elif "thời tiết" in text:
                current_weather()
            elif "youtube" in text:
                youtube()
            elif "hình nền" in text:
                change_wallpaper()
            elif "đọc báo" in text:
                read_news()
            elif "wiki" in text:
                tell_me_about()
            else:
                speak("Tôi chưa có chức năng này")

# DEMO 

In [21]:
assistant()

Vias: Xin chào, Tôi là Vias
            Bạn tên là gì?
Tôi: alo alo 1 2 3
Vias: Chào alo alo 1 2 3
Vias: Tôi có thể giúp gì?
Tôi: YouTube
Vias: Bạn muốn xem gì?
Tôi: Android
Vias: Video đang được mở trên Youtube.



    Error 263 for command:
        close voice01102021151304.mp3
    The specified device is not open or is not recognized by MCI.
Failed to close the file: voice01102021151304.mp3


Tôi: ...
Vias: Vias không nghe rõ. Bạn nói lại được không!



    Error 263 for command:
        close voice01102021151314.mp3
    The specified device is not open or is not recognized by MCI.
Failed to close the file: voice01102021151314.mp3


Tôi: thời gian
Vias: Tôi chưa có chức năng này


...


    Error 263 for command:
        close voice01102021151418.mp3
    The specified device is not open or is not recognized by MCI.
Failed to close the file: voice01102021151418.mp3


Tôi: xem thời gian và ngày mấy
Vias: Hôm nay là ngày 1 tháng 10 năm 2021



    Error 263 for command:
        close voice01102021151428.mp3
    The specified device is not open or is not recognized by MCI.
Failed to close the file: voice01102021151428.mp3


Tôi: kết thúc
Vias: Tạm biệt



    Error 263 for command:
        close voice01102021151441.mp3
    The specified device is not open or is not recognized by MCI.
Failed to close the file: voice01102021151441.mp3


# GUI DEMO

In [ ]:
from tkinter import *
import tkinter as tk

from tkinter import font

Window = Tk()        

# The main layout of the chat
        # to show chat window
Window.deiconify()
Window.resizable(width = False,
                              height = False)
Window.configure(width = 470,
                              height = 650,
                              bg = "#17202A")
labelHead = Label(Window,
                             bg = "#17202A",
                              fg = "#EAECEE",
                              text = " V I E T V I A S " ,
                               font = "Ubuntu 19 ",
                               pady = 5)
labelHead.place(relwidth = 1)

line = Label(Window,
                          width = 450,
                          height=550,
                          bg = "#ABB2B9")
         
line.place(relwidth = 1,
                        rely = 0.07,
                        relheight = 0.012)
         
textCons = Text(Window,
                             width = 20,
                             height = 2,
                             bg = "#17202A",
                             fg = "#EAECEE",
                             font = "Ubuntu 10 ",
                             padx = 5,
                             pady = 5)
         
textCons.place(relheight = 0.91,
                            relwidth = 1,
                            rely = 0.08)


textCons.config(cursor = "arrow")

                # create a scroll bar
scrollbar = Scrollbar(textCons)
         
        # place the scroll bar
        # into the gui window
scrollbar.place(relheight = 1,
                        relx = 0.974)
         
scrollbar.config(command = textCons.yview)

textCons.config(state = DISABLED)
Window.mainloop()
